# Create non-taxa csv

Create a csv that contain non-taxa coulmns.

In [1]:
import sys
sys.path.append('../scripts/')
import glob
import re
import os.path

import pandas as pd
import numpy as np

from normalize_data import (
    csv_cleanup,
    update_metadata,
    get_taxonomy_columns,
)

In [2]:
non_taxa_fields_path = 'cleaned_data/taxa/non_taxa_fields.csv'

micropal_1 = 'cleaned_data/Micropal_CSV_1'
micropal_meta_1 = 'cleaned_data/metadata/Micropal_1_changes.csv'

micropal_2 = 'cleaned_data/Micropal_CSV_2'
micropal_meta_2 = 'cleaned_data/metadata/Micropal_2_changes.csv'

micropal_3 = 'cleaned_data/Micropal_CSV_3'
micropal_meta_3 = 'cleaned_data/metadata/Micropal_3_changes.csv'

In [3]:
metadata_file = micropal_meta_3
clean_data_path = micropal_3

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True,False,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True,False,True
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True,False,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True,False,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False,False,True


## Get non-taxa columns

The Micropal CSVs have non-taxa columns, then taxa columns, then non-taxa columns. We want to form a set of non-taxa columns.

Get the column names from the beginning and end of the header row. Manually inspect the columns to select the non-taxa columns.

In [5]:
sample_columns = set()

sample_columns.update([
'Sample',
'Exp',
'Site',
'Hole',
'Core',
'Core-Sect',
'Extra Sample ID Data',
'Type',
'Section',
'A/W',
'Top [cm]',
'Bottom [cm]',
'Top Depth [m]',
'Bottom Depth [m]', 
])
len(sample_columns)

14

In [6]:
taxa_metadata_columns = set()

taxa_metadata_columns.update([
'% Planktic Foraminifera within whole sample',
'Abundance',
'Additional zone name',
'Additional zone name (short)',
'Age',
'Aspect comment (etching)',
'Aspect comment (etching),Comment (general)',
'BF Group abundance',
'BF Preservation',
'BF comment',
'BF preservation',
'COMMENTS',
'Chrysophyte cyst group abundance',
'Comment',
'Comment (general)',
'Comments',
'Datum age average [Ma]',
'Datum age maximum [Ma]',
'Datum age minimum [Ma]',
'Datum author year',
'Datum comment',
'Datum group',
'Datum group code',
'Datum name',
'Datum name generic',
'Datum occurrence',
'Datum region',
'Datum status',
'Datum type',
'Datum validation comment',
'Diatom abundance',
'Diatom preservation - pyritization2',
'Diatom preservation dissolution',
'Diatom preservation fragmentation',
'Diatoms and siliceous plankton comment',
'Diatoms group abundance',
'Ebridian group abundance',
'File Data',
'Foram abundance',
'Fragmentation',
'Fragmentation rank [auto-pop]',
'General comment',
'Genus/species (upper zone)',
'Genus/species lower zone)',
'Group Abundance',
'Group abundance',
'Group abundance (%)',
'Group preservation',
'Large Benthic Forams [%]',
'Lower boundary age av. [Ma]',
'Lower boundary age max [Ma]',
'Lower boundary age min [Ma]',
'Mixing',
'Nannofossil abundance',
'Nannofossil comment',
'No. specimens/tray',
'PALEO WATER DEPTH (IS=inner shelf, MS=middle shelf, OS=outer shelf)',
'PF Group Abundance',
'PF Preservation',
'PF Zone',
'PF group abundance',
'PF preservation',
'Percentage of benthic forams in total foram assemblage [%]',
'Percentage of non-calcareous agglutinated forams in total foram assemblage [%]',
'Percentage of planktic forams in total foram assemblage [%]',
'Piece',
'Planktonic Benthic ratio (P:B)',
'Pleurostomellids comment',
'Preservation',
'Pteropod group abundance',
'Reworking comment (1= <1%, 2= light 1-10%, 3= >10%)',
'Reworking comment (1= <1%, 2=light 1-10%, 3= >10%)',
'Sample comment',
'Sample preparation comment',
'Ship File Links',
'Shore File Links',
'Sillicoflagellate abundance',
'Silicoflagellates group abundance',
'Temperature Range',
'Total in situ dinocysts',
'Type (lower zone)',
'Type (upper zone)',
'Upper boundary age av. [Ma]',
'Upper boundary age max [Ma]',
'Upper boundary age min [Ma]',
'XBroken',
'XCorroded',
'XCrumpled',
'Zone',
'Zone author (year)',
'Zone comment',
'Zone group',
'Zone group,Type (upper zone)',
'Zone name',
'Zone name (short)',
'Zone name [short]',
'Zone status',
'comments',
'constituent',
'count',
'count_type',
'dupes and comments',
'pc_abundance_name_mode',
'pc_fossil_group',
'pc_fossil_name',
'pc_preservation_name_average',
'physical_constituent_name'
])

len(taxa_metadata_columns)

107

In [7]:
non_taxa_columns = set()
non_taxa_columns.update(sample_columns)
non_taxa_columns.update(taxa_metadata_columns)

Get all the columns from the beginning of the file.

In [8]:
filtered_columns = set()

for file in metadata['file']:
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    content = content.dropna(axis="columns", how="all")
    columns = get_taxonomy_columns(content.columns, non_taxa_columns) 
    filtered_columns.update(set(columns[0:5]))

filtered_columns

{' Circodiscus (Porodiscus) circularis',
 ' Cycladophora sakaii',
 ' Lithelius barbatus',
 'Abyssamina poagi',
 'Abyssamina quadrata',
 'Acarinina aspensis',
 'Acarinina bullbrooki',
 'Acarinina coalingensis',
 'Acarinina collactea',
 'Acarinina cuneicamerata',
 'Acarinina densa ',
 'Acarinina mcgowrani',
 'Acarinina medizzai ',
 'Acarinina praetopilensis ',
 'Acarinina soldadoensis',
 'Acarinina topilensis',
 'Acarinina triplex',
 'Acarinina wilcoxensis',
 'Achnanthes inflata',
 'Achnanthes lanceolata',
 'Achnanthes spp.',
 'Acrocubus octopylus',
 'Acrosphaera murrayana',
 'Acrosphaera sp.',
 'Acrosphaera spinosa',
 'Acrosphaera trepanata',
 'Actinocyclus curvatulus',
 'Actinocyclus ellipticus',
 'Actinocyclus ellipticus fa. lanceolata',
 'Actinocyclus elongatus',
 'Actinocyclus ingens f. nodus',
 'Actinocyclus ochotensis',
 'Actinocyclus octonarius',
 'Actinocyclus oculatus',
 'Actinocyclus spp.',
 'Actinomma boreale',
 'Actinomma delicatulum ',
 'Actinomma leptodermum',
 'Actinomma 

Get all the columns from the end of the file.

In [9]:
filtered_columns = set()

for file in metadata['file']:
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
 
    content = content.dropna(axis="columns", how="all")
    columns = get_taxonomy_columns(content.columns, non_taxa_columns)            
    filtered_columns.update(set(columns[-5:]))
    
filtered_columns

{'Acarinina coalingensis',
 'Acarinina soldadoensis',
 'Agglutinated',
 'Agglutinated gen. et sp. indet.',
 'Amphistegina lessonii',
 'Anomalinoides spp.',
 'Asterigerina planorbis',
 'Asterigerina sp.',
 'Bolivina cf. robusta',
 'Botrychium type',
 'Brassicaceae',
 'Brizalina alata',
 'Brizalina dilatata',
 'Brizalina spp.',
 'Broinsonia parca constricta',
 'Bulimina aculeata',
 'Bulimina cf. alazanensis',
 'Bulimina elongata',
 'Bulimina gibba',
 'Bulimina marginata',
 'Bulimina spp.',
 'Bulimina striata gr.',
 'Buliminella elegantissima',
 'Calcareous gen. et sp. indet.',
 'Calcidiscus macintyrei',
 'Carpocanopsis cingulata',
 'Carpocanopsis cristata',
 'Caryophyllaceae',
 'Ceratolithus cristatus',
 'Chiasmolithus grandis',
 'Cibicides refulgens',
 'Cibicides/Cibicidoides spp.',
 'Cibicidoides cf. wuellerstorfi',
 'Cibicidoides kullenbergi',
 'Cibicidoides pachyderma',
 'Cibicidoides wuellerstorfi',
 'Coccolithus miopelagicus',
 'Coccolithus pelagicus',
 'Conorboides',
 'Cribrostomo

## Create non-taxa csv

In [10]:
df = pd.DataFrame()
df['field'] = list(taxa_metadata_columns)
df['type'] = 'taxonomy metadata'

In [11]:
df2 = pd.DataFrame()
df2['field'] = list(sample_columns)
df2['type'] = 'sample'

In [12]:
df = df.append(df2, ignore_index=True)

In [13]:
df.shape

(121, 2)

In [14]:
df.sort_values(['type','field'], inplace=True)
df.head()

,field,type
111,A/W,sample
117,Bottom Depth [m],sample
115,Bottom [cm],sample
110,Core,sample
107,Core-Sect,sample


In [15]:
df.to_csv(non_taxa_fields_path, index=False)